In [9]:
from pdf2image import convert_from_path
from PIL import Image, ImageEnhance
from fpdf import FPDF


def adjust_contrast(image, contrast_factor):
    enhancer = ImageEnhance.Contrast(image)
    adjusted_image = enhancer.enhance(contrast_factor)
    return adjusted_image


def desaturate(image):
    return image.convert("L")


def adjust_sharpness(image, sharpness_factor):
    enhancer = ImageEnhance.Sharpness(image)
    adjusted_image = enhancer.enhance(sharpness_factor)
    return adjusted_image


# lower temperature = more blue / cold
# higher temperature = more red / yellow / warm
def adjust_temperature(image, temperature):
    # Define color balance adjustments
    red_ratio = 1.0
    green_ratio = 1.0
    blue_ratio = max(0, 1.0 - (temperature / 100.0))

    # Apply color balance adjustments
    cooler_image = Image.new("RGB", image.size)
    pixels = cooler_image.load()

    for x in range(image.width):
        for y in range(image.height):
            r, g, b = image.getpixel((x, y))
            new_r = int(r * red_ratio)
            new_g = int(g * green_ratio)
            new_b = int(b * blue_ratio)
            pixels[x, y] = (new_r, new_g, new_b)

    return cooler_image


def process_pdf_with_adjusted_contrast(pdf_path, contrast_factor):
    images = convert_from_path(pdf_path)

    adjusted_images = []
    for i, image in enumerate(images):
        adjusted_image = adjust_sharpness(image, sharpness_factor)
        adjusted_image = adjust_temperature(image, -50)
        adjusted_image = adjust_contrast(image, contrast_factor)
        adjusted_image = desaturate(adjusted_image)
        image_name = f"adjusted_page_{i+1}.png"
        adjusted_image.save(image_name)
        adjusted_images.append(image_name)

    return adjusted_images


def create_pdf_from_images(image_paths, output_pdf_path):
    pdf = FPDF()
    for image_path in image_paths:
        pdf.add_page()
        pdf.image(
            image_path, x=0, y=0, w=210, h=297
        )  # Adjust width and height as needed
    pdf.output(output_pdf_path)


pdf_path = "input/4.pdf"
contrast_factor = 2.0  # Adjust this value to control contrast (1.0 is default)
sharpness_factor = 2.0  # Adjust this value to control sharpness (1.0 is default)
adjusted_images = process_pdf_with_adjusted_contrast(pdf_path, contrast_factor)

output_pdf_path = "output/adjusted_output.pdf"
create_pdf_from_images(adjusted_images, output_pdf_path)